In [403]:
import datetime

import numpy as np
import pandas as pd
from talib import abstract

import warnings
warnings.filterwarnings('ignore')


SMA = abstract.SMA


In [404]:
def load_data(file_path):
    data = pd.read_csv(file_path)
    # data['Close'] = data['Price']
    data['Open'] = data['Open']
    data['Close'] = data['Close']
    data['High'] = data['High']
    data['Low'] = data['Low']
    return data

In [405]:
# Load historical market data
data = load_data("data2.csv")
data.head()

,Date,Open,High,Low,Close
0,2000-01-01 00:00:00,36063,69908,60628,67354
1,2000-01-02 00:00:00,48763,29568,18146,87904
2,2000-01-03 00:00:00,37359,99678,8815,65351
3,2000-01-04 00:00:00,60449,8657,47729,67330
4,2000-01-05 00:00:00,21278,99803,85125,76529


In [406]:
from abc import ABC, abstractmethod


class Strategy(ABC):

    def __init__(self):
        pass
    
    @abstractmethod
    def generate_signal(self, data):
        pass


In [407]:
class SMAStrategy(Strategy):
    def __init__(self, short_window, long_window, sl, tp1, tp2, tp3):
        super().__init__()
        self.short_window = short_window
        self.long_window = long_window
        self.stop_loss = sl
        self.take_profit1 = tp1
        self.take_profit2 = tp2
        self.take_profit3 = tp3

    #####################################################################################################
    # Signal
    #####################################################################################################

    def generate_signal(self, data):
        signals = pd.Series(index=data.index, dtype='int')

        data['Short_MA'] = SMA(data, timeperiod=self.short_window, price='Open')
        data['Long_MA'] = SMA(data, timeperiod=self.long_window, price='Open')

        # Buy signal
        buy_signal = (data['Short_MA'] > data['Long_MA']) #& (signals.shift(1) != 1)
        signals[buy_signal] = 1

        # Sell signal
        sell_signal = (data['Short_MA'] < data['Long_MA']) #& (signals.shift(1) != -1)
        signals[sell_signal] = -1
        
    #####################################################################################################
    # Set Stoploss TakeProfit
    #####################################################################################################

        # Apply stop-loss and take-profit
        data['Signal'] = signals
        data['Sl'] = np.where(signals == 1, data['Open'] * (1 - self.stop_loss), data['Open'] * (1 + self.stop_loss))
        data['Tp1'] = np.where(signals == 1, data['Open'] * (1 + self.take_profit1), data['Open'] * (1 - self.take_profit1))
        data['Tp2'] = np.where(signals == 1, data['Open'] * (1 + self.take_profit2), data['Open'] * (1 - self.take_profit2))
        data['Tp3'] = np.where(signals == 1, data['Open'] * (1 + self.take_profit3), data['Open'] * (1 - self.take_profit3))

        # data = data.drop(['Short_MA', 'Long_MA', "Vol.", "Change %", "Price"], axis=1)
        data = data.dropna()
        return data

sma_strategy = SMAStrategy(2, 5, 0.05, 0.01, 0.015, 0.02)
generated_signals = sma_strategy.generate_signal(data)

generated_signals

,Date,Open,High,Low,Close,Short_MA,Long_MA,Signal,Sl,Tp1,Tp2,Tp3
4,2000-01-05 00:00:00,21278,99803,85125,76529,40863.5,40782.4,1.0,20214.10,21490.78,21597.170,21703.56
5,2000-01-06 00:00:00,44945,64440,24247,46338,33111.5,42558.8,-1.0,47192.25,44495.55,44270.825,44046.10
6,2000-01-07 00:00:00,1525,81761,48916,20722,23235.0,33111.2,-1.0,1601.25,1509.75,1502.125,1494.50
7,2000-01-08 00:00:00,20509,93595,40083,85981,11017.0,29741.2,-1.0,21534.45,20303.91,20201.365,20098.82
8,2000-01-09 00:00:00,59570,24327,19537,78719,40039.5,29565.4,1.0,56591.50,60165.70,60463.550,60761.40
...,...,...,...,...,...,...,...,...,...,...,...,...
711,2001-12-12 00:00:00,27922,34842,67733,15546,54954.0,50971.2,1.0,26525.90,28201.22,28340.830,28480.44
712,2001-12-13 00:00:00,58434,83204,29400,52950,43178.0,53662.0,-1.0,61355.70,57849.66,57557.490,57265.32
713,2001-12-14 00:00:00,34692,27357,58721,47724,46563.0,44696.0,1.0,32957.40,35038.92,35212.380,35385.84
714,2001-12-15 00:00:00,69499,78422,25578,82906,52095.5,54506.6,-1.0,72973.95,68804.01,68456.515,68109.02


# First Version on Main Function

In [408]:
trade_log = pd.DataFrame(columns=['Open DateTime', 'Open Price', 
                                  'Close DateTime', 'Close Price',
                                  'Direction',
                                  'Leverage', 
                                  'Margin', 
                                  'Result',
                                  'Profit Loss Percentage', 
                                  'profit Loss Amount',
                                  ]
                         )


def open_position(direction, open_price, leverage, margin):
    global trade_log
    print('Balance Management')
    
    print('open **********************', direction, open_price, leverage, margin)
    
    open_datetime = datetime.datetime.now()
    
    # New row
    trade_log.loc[len(trade_log)] = [open_datetime, open_price, 0, 0, direction, leverage, margin, 'None', 0, 0]
    

    
def close_position(close_price):
    global trade_log
    
    print('close **********************', close_price)
    
    # Update last row
    close_datetime = datetime.datetime.now()
    
    trade_log['Close DateTime'].iloc[-1] = close_datetime
    trade_log['Close Price'].iloc[-1] = close_price
    
    # Calculate Profit
    calculate_profit_loss()
    


def calculate_profit_loss():
    global trade_log
    
    # calculate last row
    open_price = trade_log['Open Price'].iloc[-1]
    close_price = trade_log['Close Price'].iloc[-1]
    direction = trade_log['Direction'].iloc[-1]
    leverage = trade_log['Leverage'].iloc[-1]
    margin = trade_log['Margin'].iloc[-1]
    
    
    print('calculate *****************', open_price, close_price, direction, leverage, margin)
    
        
    if direction == 'long':
        result = close_price - open_price
    elif direction == 'short':
        result = open_price - close_price
    else:
        raise ValueError("Invalid transaction type. Must be 'long' or 'short'.")
    
    profit_loss_percentage = (result / open_price) * 100
    profit_loss_amount = result * margin * leverage
    
    if result > 0:
        profit_loss_status = 'profit'
    else:
        profit_loss_status = 'loss'
    

    trade_log['Result'].iloc[-1] = profit_loss_status
    trade_log['Profit Loss Percentage'].iloc[-1] = profit_loss_percentage
    trade_log['profit Loss Amount'].iloc[-1] = profit_loss_amount
    


trade_log

,Open DateTime,Open Price,Close DateTime,Close Price,Direction,Leverage,Margin,Result,Profit Loss Percentage,profit Loss Amount


In [409]:
history = pd.DataFrame(columns=['Direction', 'Sl', 'Tp1', 'Tp2', 'Tp3', 'Status'])
history.loc[0] = [0, 0, 0, 0, 0, 'None']

current_position = 0
previous_position = 0


def check_tp_sl(new_data):
    global history, current_position
    
    sl = history['Sl'].iloc[-1]
    tp1 = history['Tp1'].iloc[-1]
    tp2 = history['Tp2'].iloc[-1]
    tp3 = history['Tp3'].iloc[-1]
    
    low = new_data['Low']
    high = new_data['High']
    
    
    if current_position == 1:
        if low < sl :
            print('low ', low,' sl', sl)
            print("SL Active Long")
            
            close_position(sl)
            
            return 1, 'loss'
        
        if high > tp1 or high > tp2  or high > tp3:
            print('high ', high, ' tp ', tp1, tp2, tp3)
            print("TP Active Long")
            


            if high > tp3 and high > tp2 and high > tp3:
                close_position(tp3)
            elif high > tp1 and high > tp2:
                close_position(tp2)  
            elif high > tp1:
                close_position(tp1)
                
            # close_position(tp1)
            
            return 1, 'profit'


    elif current_position == -1 :
        if high > sl:
            print('high ', high,' sl', sl)
            print("SL Active SELL")
            
            close_position(sl)
            
            return -1, 'loss'
        
        if low < tp1 or low < tp2 or low < tp3:
            print('low ', low, ' tp ', tp1, tp2, tp3)
            print("TP Active SELL")
            

            if low < tp3 and low < tp2 and low < tp3:
                close_position(tp3)
            elif low < tp1 and low < tp2:
                close_position(tp2)
            elif low < tp1:
                close_position(tp1)

            # close_position(tp1)
            
            return -1, 'profit'

    return None, None


In [410]:
def refresh(order):
    global history, current_position, previous_position, trade_log
    
    signal = order['Signal']
    price = order['Open']     
    low = order['Low']
    high = order['High']
    sl = order['Sl']
    tp1 = order['Tp1']
    tp2 = order['Tp2']
    tp3 = order['Tp3']


    print('*************** new data received ***************')   
    print(previous_position, 'prev')
    print(current_position, 'curr')
    print('open', price)
    print('low', low)
    print('high', high)
    print('sl', sl)
    print('tp1', tp1)
    print('tp2', tp2)
    print('tp3', tp3)
    print('*************************************************')


      #####################################################################################################
      # Check Sl Tp
      #####################################################################################################

    if current_position != 0 :
        
        # Check Tp Sl and save the data
        direction, status = check_tp_sl(order)
        
        # Check profit or loss and save to history
        if status :
            history['Status'].iloc[-1] = status
            current_position = 0
            previous_position = direction
            return None
        
        
      #####################################################################################################
      # Open or Close Position
      #####################################################################################################

       
    if signal != np.nan:
        print("Step: " , i, "      Signal is : " , signal)


        #open long
        if signal == 1 and current_position == 0 and previous_position != 1:
            print("Open Long Position")
            
            open_position('long', price, 1, 10)
            
            current_position = signal
            history.loc[len(history)] = [signal, sl, tp1, tp2, tp3, '']


        #Close short, open long
        elif signal == 1 and current_position == -1:
            print("Close Short Position")
            print("Open Long Position")
            
            close_position(price)
            open_position('long', price, 1, 10)
            
            history['Status'].iloc[-1] = 'Direction Changed'
            current_position = signal
            history.loc[len(history)] = [signal, sl, tp1, tp2, tp3, '']



        # open short
        elif signal == -1 and current_position == 0 and previous_position != -1:
            print("Open Short Position")
            
            open_position('short', price, 1, 10)
            
            current_position = signal
            history.loc[len(history)] = [signal, sl, tp1, tp2, tp3, '']


        # close long, open short
        elif signal == -1 and current_position == 1:
            print("Close Long Position")
            print("Open Short Position")
            
            close_position(price)
            open_position('short', price, 1, 10)
            
            history['Status'].iloc[-1] = 'Direction Changed'
            current_position = signal
            history.loc[len(history)] = [signal, sl, tp1, tp2, tp3, '']



# Call refresh by every new data
for i in range(len(generated_signals)):
    new_data = generated_signals.iloc[i]
    
    refresh(new_data)
      

history

*************** new data received ***************
0 prev
0 curr
open 21278
low 85125
high 99803
sl 20214.1
tp1 21490.78
tp2 21597.17
tp3 21703.56
*************************************************
Step:  0       Signal is :  1.0
Open Long Position
Balance Management
open ********************** long 21278 1 10
*************** new data received ***************
0 prev
1.0 curr
open 44945
low 24247
high 64440
sl 47192.25
tp1 44495.55
tp2 44270.825
tp3 44046.1
*************************************************
high  64440  tp  21490.78 21597.17 21703.56
TP Active Long
close ********************** 21703.56
calculate ***************** 21278 21703.56 long 1 10
*************** new data received ***************
1 prev
0 curr
open 1525
low 48916
high 81761
sl 1601.25
tp1 1509.75
tp2 1502.125
tp3 1494.5
*************************************************
Step:  2       Signal is :  -1.0
Open Short Position
Balance Management
open ********************** short 1525 1 10
*************** new data received

,Direction,Sl,Tp1,Tp2,Tp3,Status
0,0.0,0.00,0.00,0.000,0.00,None
1,1.0,20214.10,21490.78,21597.170,21703.56,profit
2,-1.0,1601.25,1509.75,1502.125,1494.50,loss
3,1.0,56591.50,60165.70,60463.550,60761.40,loss
4,-1.0,88381.65,83331.27,82910.405,82489.54,profit
...,...,...,...,...,...,...
228,-1.0,17147.55,16167.69,16086.035,16004.38,loss
229,1.0,74398.30,79097.14,79488.710,79880.28,profit
230,-1.0,21468.30,20241.54,20139.310,20037.08,loss
231,1.0,26525.90,28201.22,28340.830,28480.44,profit
